In [10]:
!*rm -f go.work && go work init && go work use . "${HOME}/Projects/gomlx"
%goworkfix

	- Replace rule for module "github.com/gomlx/gomlx" to local directory "/home/janpf/Projects/gomlx" already exists.


## Downloading Dataset

The method `mag.Download()` will download the dataset to the data directory, if not yet downloaded.
It then converts the dataset to tensors, which are then available for use.

The tensor is then saved for faster access. After saved, the next call to `mag.Download()` will take 1/2s.

In [2]:
%env GOMLX_PLATFORM=GPU

Set: GOMLX_PLATFORM="GPU"


In [16]:
import (
    "flag"

    // . "github.com/gomlx/gomlx/graph"
    // "github.com/gomlx/gomlx/types/shapes"
    mag "github.com/gomlx/gomlx/examples/ogbnmag"
    "github.com/janpfeifer/must"
)

var (
    flagDataDir   = flag.String("data", "~/work/ogbnmag", "Directory to cache downloaded and generated dataset files.")
)

%%
start := time.Now()
must.M(mag.Download(*flagDataDir))
fmt.Printf("Elapsed: %s\n", time.Since(start))

Elapsed: 558.160595ms


## FNN model, No Graph

The first model will only use the paper features, and no relations. It serves as a baseline.

In a quick experiment, without much hyperparameter tuning, we got 27.27% on test accuracy, which is inline with the corresponding results in [the leaderboard](https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-mag) (the Multi Layer Perceptron MLP entry in the bottom).

In [34]:
import (
	. "github.com/gomlx/gomlx/graph"
	"github.com/gomlx/gomlx/ml/context"
)
var manager = NewManager()

func config() (ctx *context.Context) {
    ctx = context.NewContext(manager)
    ctx.RngStateReset()
    ctx.SetParams(map[string]any{
        "train_steps": 2_000_000, 
        "batch_size": 128,
        "optimizer": "adamw", 
        optimizers.LearningRateKey: 0.0001,
        layers.L2RegularizationKey: 1e-4,
        "normalization": "layer",
        "dropout": 0.1,
        "hidden_layers": 2,
        "num_nodes": 256,
        "plots": true,
    })
    return
}

In [35]:
import "github.com/gomlx/gomlx/examples/ogbnmag/fnn"

%%
must.M(mag.Download(*flagDataDir))
ctx := config()
ctx.SetParam("checkpoint", path.Join(*flagDataDir, "fnn-baseline"))
err := fnn.Train(ctx)
if  err != nil {
    fmt.Printf("%+v\n", err)
}


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.676684e+00 L8.493124e+01,2.336741e+01 L1.261747e+02,2.829207e+01 L1.567907e+02,7.310011e+01 L1.815883e+02,8.945928e+01 L2.028195e+02,9.427825e+01 L2.215522e+02,1.148646e+02 L2.385076e+02,9.776568e+01 L2.541305e+02,1.229385e+02 L2.687114e+02,1.220508e+02 L2.824483e+02,1.312449e+02 L2.954812e+02,1.311181e+02 L3.079489e+02,1.272502e+02 L3.199465e+02,1.283704e+02 L3.315401e+02,1.306954e+02 L3.427765e+02,1.336755e+02 L3.537117e+02,1.372263e+02 L3.643830e+02,1.418974e+02 L3.748325e+02,1.408617e+02 L3.850867e+02,1.404390e+02 L3.951751e+02,1.440532e+02 L4.051145e+02,1.528457e+02 L4.149242e+02,1.593767e+02 L4.246136e+02,1.595881e+02 L4.341943e+02,1.513874e+02 L4.436785e+02,1.621667e+02 L4.530776e+02,1.625471e+02 L4.624015e+02,1.625260e+02 L4.716581e+02,1.590597e+02 L4.808533e+02,1.616171e+02 L4.899960e+02,1.601588e+02 L4.990914e+02,1.723330e+02 L5.081417e+02,1.675986e+02 L5.171541e+02,1.718258e+02 L5.261313e+02,1.623780e+02 L5.350754e+02,1.673450e+02 L5.439907e+02,1.725444e+02 L5.528792e+02,1.810199e+02 L5.617455e+02,1.673450e+02 L5.705911e+02,1.771097e+02 L5.794184e+02,1.814003e+02 L5.882283e+02,1.797729e+02 L5.970230e+02,1.907424e+02 L6.058036e+02,1.809565e+02 L6.145711e+02,1.832391e+02 L6.233272e+02,1.794135e+02 L6.320730e+02,1.889881e+02 L6.408090e+02,1.906790e+02 L6.495363e+02,1.877833e+02 L6.582558e+02,1.786315e+02 L6.669682e+02,1.910594e+02 L6.756739e+02,1.820767e+02 L6.843732e+02,1.928348e+02 L6.930673e+02,1.862827e+02 L7.017566e+02,1.967027e+02 L7.104411e+02,1.985626e+02 L7.191214e+02,1.921373e+02 L7.277979e+02,1.996617e+02 L7.364712e+02,1.994715e+02 L7.451414e+02,2.009087e+02 L7.538088e+02,1.984147e+02 L7.624737e+02,1.924332e+02 L7.711364e+02,1.906367e+02 L7.797968e+02,2.002324e+02 L7.884555e+02,1.998731e+02 L7.971123e+02,1.985626e+02 L8.057675e+02,1.995983e+02 L8.144213e+02,1.991967e+02 L8.230738e+02,1.957938e+02 L8.317250e+02,2.012046e+02 L8.403751e+02,2.007396e+02 L8.490243e+02,2.054741e+02 L8.576726e+02,2.031069e+02 L8.663200e+02,1.959207e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.785369e+01 L8.493124e+01,5.949159e+01 L1.261747e+02,5.937473e+01 L1.567907e+02,8.178258e+01 L1.815883e+02,8.681748e+01 L2.028195e+02,9.805167e+01 L2.215522e+02,1.128903e+02 L2.385076e+02,1.186981e+02 L2.541305e+02,1.264716e+02 L2.687114e+02,1.309602e+02 L2.824483e+02,1.359261e+02 L2.954812e+02,1.433517e+02 L3.079489e+02,1.443992e+02 L3.199465e+02,1.528837e+02 L3.315401e+02,1.557714e+02 L3.427765e+02,1.609232e+02 L3.537117e+02,1.642221e+02 L3.643830e+02,1.681714e+02 L3.748325e+02,1.698596e+02 L3.850867e+02,1.714337e+02 L3.951751e+02,1.743862e+02 L4.051145e+02,1.757815e+02 L4.149242e+02,1.789987e+02 L4.246136e+02,1.828101e+02 L4.341943e+02,1.840716e+02 L4.436785e+02,1.867975e+02 L4.530776e+02,1.882617e+02 L4.624015e+02,1.903469e+02 L4.716581e+02,1.919492e+02 L4.808533e+02,1.947046e+02 L4.899960e+02,1.950270e+02 L4.990914e+02,1.982062e+02 L5.081417e+02,2.008743e+02 L5.171541e+02,2.015248e+02 L5.261313e+02,2.027610e+02 L5.350754e+02,2.049448e+02 L5.439907e+02,2.069159e+02 L5.528792e+02,2.080817e+02 L5.617455e+02,2.082380e+02 L5.705911e+02,2.105584e+02 L5.794184e+02,2.118157e+02 L5.882283e+02,2.132264e+02 L5.970230e+02,2.146288e+02 L6.058036e+02,2.164986e+02 L6.145711e+02,2.178559e+02 L6.233272e+02,2.181839e+02 L6.320730e+02,2.199129e+02 L6.408090e+02,2.217602e+02 L6.495363e+02,2.223177e+02 L6.582558e+02,2.229668e+02 L6.669682e+02,2.235342e+02 L6.756739e+02,2.260587e+02 L6.843732e+02,2.273766e+02 L6.930673e+02,2.290647e+02 L7.017566e+02,2.292097e+02 L7.104411e+02,2.308359e+02 L7.191214e+02,2.327592e+02 L7.277979e+02,2.327944e+02 L7.364712e+02,2.341024e+02 L7.451414e+02,2.338926e+02 L7.538088e+02,2.351218e+02 L7.624737e+02,2.361468e+02 L7.711364e+02,2.369141e+02 L7.797968e+02,2.389092e+02 L7.884555e+02,2.400905e+02 L7.971123e+02,2.407917e+02 L8.057675e+02,2.423996e+02 L8.144213e+02,2.417801e+02 L8.230738e+02,2.441779e+02 L8.317250e+02,2.447748e+02 L8.403751e+02,2.4490

> papers features input shape: (Float32)[128 132]
Training (2000000 steps):   63% [========================>...............] (2088 steps/s) 22s:5m45s] [loss=2.761] [~loss=2.917] [~acc=30.89%]        

^C
signal: interrupt
